# K-Means

In [3]:
import numpy as np
import tensorflow as tf

In [35]:
#Load Data
def loadDataSet(filename):
    dataMat = []
    fr = open(filename)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = map(float, curLine)
        dataMat.append(fltLine)
    return dataMat

#Compting the Euclidean Distance
def distEuclide(vecA, vecB):
    return tf.sqrt(tf.reduce_sum(tf.square(vecA - vecB), 2))

#Choose Random Centroids
def initRandomCentroid(dataSet, k):
    m = tf.shape(dataSet)[0]
    randIndices = tf.random_shuffle(tf.range(0, m))
    begin = [0,]
    size = [k,]
    size[0] = k
    centroidIndices = tf.slice(randIndices, begin, size)
    initCentroids = tf.gather(dataSet, centroidIndices)
    return initCentroids

#Assign Each sample to the Nearest Centroid
def assignToNearest(dataSet, centroids, distMeans):
    #Choose Euclidean Distance
    expandVectors = tf.expand_dims(dataSet, 0)
    expandCentroids = tf.to_float(tf.expand_dims(centroids, 1))
    distance = distMeans(expandVectors, expandCentroids)
    mins = tf.argmin(distance, 0)
    nearestIndices = mins

    return nearestIndices

def updateCentroids(dataSet, nearestIndices, k):
    nearestIndices = tf.to_int32(nearestIndices)
    partitions = tf.dynamic_partition(dataSet, nearestIndices, k)
    newCentroids = [tf.reduce_mean(partition, 0) for partition in partitions]
    
    return newCentroids

def kMeans(dataSet, k, distMeans = distEuclide, createCent = initRandomCentroid):
    initCent = createCent(dataSet, k)
    initNearestIndices = assignToNearest(dataSet, initCent, distMeans)
    preIndices = initNearestIndices
    newCent = updateCentroids(dataSet, initNearestIndices, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        nearestIndices = assignToNearest(dataSet, newCent, distMeans)
        newCent = updateCentroids(dataSet, nearestIndices, k)
        print newCent
        flag = tf.cast(tf.equal(preIndices,nearestIndices), tf.float32)
        if flag[tf.argmin(flag,0)] == 0:
            clusterChanged = True
    print "Done"
    return newCent

In [36]:
Y = tf.placeholder(tf.float32,[None, 2])
centroid = kMeans(Y, 4)

with tf.Session() as sess:
    dataSet = loadDataSet('testSet.tt')
    finalCent = sess.run(centroid, feed_dict = {Y: dataSet})
    print finalCent

[<tf.Tensor 'Mean_52:0' shape=(2,) dtype=float32>, <tf.Tensor 'Mean_53:0' shape=(2,) dtype=float32>, <tf.Tensor 'Mean_54:0' shape=(2,) dtype=float32>, <tf.Tensor 'Mean_55:0' shape=(2,) dtype=float32>]


TypeError: Bad slice index Tensor("ArgMin_18:0", shape=(), dtype=int64) of type <class 'tensorflow.python.framework.ops.Tensor'>